# Chapter 4 日本語の文章を生成しよう

## Lesson 28 日本語を形態素解析してみましょう

### 手順パート: Janomeで日本語を形態素解析する

In [1]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
t

In [2]:
text = '東京都でおいしいビールを飲もう。'
tokens = t.tokenize(text)
len(tokens)

9

In [3]:
# 文章を分割する
for token in tokens:
    print(token)

東京	名詞,固有名詞,地域,一般,*,*,東京,トウキョウ,トーキョー
都	名詞,接尾,地域,*,*,*,都,ト,ト
で	助詞,格助詞,一般,*,*,*,で,デ,デ
おいしい	形容詞,自立,*,*,形容詞・イ段,基本形,おいしい,オイシイ,オイシイ
ビール	名詞,一般,*,*,*,*,ビール,ビール,ビール
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
飲も	動詞,自立,*,*,五段・マ行,未然ウ接続,飲む,ノモ,ノモ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ
。	記号,句点,*,*,*,*,。,。,。


In [4]:
t.tokenize(text, wakati=True)

['東京', '都', 'で', 'おいしい', 'ビール', 'を', '飲も', 'う', '。']

## Lesson 29 自然言語処理で使用されるモデルやアルゴリズムを知りましょう

### 手順パート: Bag of Wordsのデータを作ってみよう

In [5]:
# 複数の文章を単語に分割する
from janome.tokenizer import Tokenizer

t = Tokenizer()  # ❶ Tokenizerのインスタンスを生成
sentences = [  # ❷ 対象となる文章のリスト
    'おいしいビールを飲む', 'コーヒーを飲む', 'おいしいクラフトビールを買う'
]

words_list = []
for sentence in sentences:
    words_list.append(t.tokenize(sentence, wakati=True))  # ❹ 文章を分かち書き
words_list

[['おいしい', 'ビール', 'を', '飲む'],
 ['コーヒー', 'を', '飲む'],
 ['おいしい', 'クラフト', 'ビール', 'を', '買う']]

In [6]:
# 一意な単語のリストを作成する
unique_words = []
for words in words_list:  # ❶各単語を取り出す
    for word in words:
        if word not in unique_words:  # ❷存在しなければ追加
            unique_words.append(word)
unique_words

['おいしい', 'ビール', 'を', '飲む', 'コーヒー', 'クラフト', '買う']

In [7]:
# Bag of Words のデータを作成する
bow_list = []
for words in words_list:
    bag_of_words = []  # ❶ 1文章のBag of Wordsを格納する
    for unique_word in unique_words:
        num = words.count(unique_word)  # ❷ 一意な単語の出現回数を数える
        bag_of_words.append(num)
    bow_list.append(bag_of_words)
bow_list

[[1, 1, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0], [1, 1, 1, 0, 0, 1, 1]]

In [8]:
# TF-IDFを計算する
from math import log  # ❶ logをインポート
num_of_sentences = len(sentences)
idf = []
for i in range(len(unique_words)):  # ❷ 一意な単語分繰り返す
    count = 0
    for bow in bow_list:  # ❸ Bag of Wordsに存在すれば1を足す
        if bow[i] > 0:
            count += 1
    idf.append(log((num_of_sentences + 1) / (count + 1)))  # ❹ 単語のIDFを計算する
idf

[0.28768207245178085,
 0.28768207245178085,
 0.0,
 0.28768207245178085,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453]

In [9]:
# TF-IDFを計算する
bow = bow_list[1]  # ❶「コーヒーを飲む」のBag of Words
num_of_words = sum(bow)  # ❷ 1文章の単語の数
tfidf = []
for i, value in enumerate(bow):
    tf = value / num_of_words  # ❸ TFを取得
    tfidf.append(tf * (idf[i] + 1))  # ❹ TF-IDFを取得
tfidf

[0.0,
 0.0,
 0.3333333333333333,
 0.42922735748392693,
 0.5643823935199818,
 0.0,
 0.0]

In [10]:
import pandas as pd

df = pd.DataFrame(bow_list, columns=unique_words)
df

,おいしい,ビール,を,飲む,コーヒー,クラフト,買う
0,1,1,1,1,0,0,0
1,0,0,1,1,1,0,0
2,1,1,1,0,0,1,1


## Lesson 32 マルコフ連鎖用の辞書データを作成しましょう

In [11]:
# 天気予報の各状態を表すデータ
weather_data = {
    '晴れ': {
        'next_weather_types': ['晴れ', '曇り', '雨'],  # 遷移先の天気のリスト
        'weights': [0.6, 0.3, 0.1],  # それぞれの天気に遷移する割合（重み）
    },
    '曇り': {'next_weather_types': ['晴れ', '曇り', '雨'], 'weights': [0.3, 0.5, 0.2]},
    '雨': {'next_weather_types': ['晴れ', '曇り', '雨'], 'weights': [0.2, 0.3, 0.5]},
}

In [12]:
# 状態を表すデータから「晴れ」の次の天気を重みありでランダムに取り出す
import random

next_weather_types = weather_data['晴れ']['next_weather_types']
weights = weather_data['晴れ']['weights']

count = {'晴れ': 0, '曇り': 0, '雨': 0}
for i in range(1000):  # 重みづけを確認するために1000回天気を取り出す
    weather = random.choices(next_weather_types, weights=weights)[0]  # リストで返るので先頭を取り出す
    count[weather] += 1  # 出現回数を数える
print(count)

{'晴れ': 581, '曇り': 307, '雨': 112}


In [13]:
# 天気のマルコフ連鎖を生成する

current_weather = '晴れ'
markov_weather_list = [current_weather]  # 天気のリストを格納する領域
for i in range(10):
    next_weather_types = weather_data[current_weather]['next_weather_types']  # 次の状態（天気）と重みを取得
    weights = weather_data[current_weather]['weights']
    current_weather = random.choices(next_weather_types, weights=weights)[0]  # 次の状態（天気）をランダムに取得
    markov_weather_list.append(current_weather)  # 天気を追加
markov_weather_list

['晴れ', '晴れ', '晴れ', '晴れ', '晴れ', '晴れ', '曇り', '雨', '晴れ', '雨', '雨']

### 手順パート: テキストからマルコフ連鎖の辞書データを作成する

In [14]:
# 1つの文を3単語ずつの組にする

from janome.tokenizer import Tokenizer

BEGIN = '__BEGIN__'  # 文の開始マーク
END = '__END__'  # 文の終了マーク

sentence = 'おいしいビールを飲もう'

t = Tokenizer()
words = t.tokenize(sentence, wakati=True)  # ❶ 文を単語に分かち書きする
words = [BEGIN] + words + [END]  # ❷ 前後に開始、終了マークを追加する

three_words_list = []
for i in range(len(words) - 2):
    three_words_list.append(words[i:i+3])
three_words_list

[['__BEGIN__', 'おいしい', 'ビール'],
 ['おいしい', 'ビール', 'を'],
 ['ビール', 'を', '飲も'],
 ['を', '飲も', 'う'],
 ['飲も', 'う', '__END__']]

In [15]:
# 複数の文章から単語の組の出現回数を数える

from collections import Counter

def get_three_words_list(sentence):  # ❶ 関数にする
    """文章を3単語の組にして返す"""
    t = Tokenizer()
    words = t.tokenize(sentence, wakati=True)
    words = [BEGIN] + words + [END]
    three_words_list = []
    for i in range(len(words) - 2):
        three_words_list.append(tuple(words[i:i+3]))
    return three_words_list

sentences = ['おいしいビールを飲もう', 'ビールを飲もう', 'おいしいビールは生']
three_words_list = []
for sentence in sentences:  # ❷ 複数の文を順番に処理
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)  # ❸ 出現回数を数える
three_words_count

Counter({('__BEGIN__', 'おいしい', 'ビール'): 2,
         ('おいしい', 'ビール', 'を'): 1,
         ('ビール', 'を', '飲も'): 2,
         ('を', '飲も', 'う'): 2,
         ('飲も', 'う', '__END__'): 2,
         ('__BEGIN__', 'ビール', 'を'): 1,
         ('おいしい', 'ビール', 'は'): 1,
         ('ビール', 'は', '生'): 1,
         ('は', '生', '__END__'): 1})

In [16]:
# 文章用の辞書データを作成する

def generate_markov_dict(three_words_count):
    """マルコフ連鎖での文章生成用の辞書データを生成する"""
    markov_dict = {}
    for three_words, count in three_words_count.items():
        two_words = three_words[:2]  # ❶ 「前半2つの単語」と「次の単語」に分割
        next_word = three_words[2]
        if two_words not in markov_dict: # ❷ 辞書に存在しない場合は空データを生成
            markov_dict[two_words] = {'words': [], 'weights': []}
        markov_dict[two_words]['words'].append(next_word)  # ❸ 次の単語と回数を追加
        markov_dict[two_words]['weights'].append(count)
    return markov_dict

markov_dict = generate_markov_dict(three_words_count)
markov_dict

{('__BEGIN__', 'おいしい'): {'words': ['ビール'], 'weights': [2]},
 ('おいしい', 'ビール'): {'words': ['を', 'は'], 'weights': [1, 1]},
 ('ビール', 'を'): {'words': ['飲も'], 'weights': [2]},
 ('を', '飲も'): {'words': ['う'], 'weights': [2]},
 ('飲も', 'う'): {'words': ['__END__'], 'weights': [2]},
 ('__BEGIN__', 'ビール'): {'words': ['を'], 'weights': [1]},
 ('ビール', 'は'): {'words': ['生'], 'weights': [1]},
 ('は', '生'): {'words': ['__END__'], 'weights': [1]}}

## Lesson 33 マルコフ連鎖で文章を自動生成しましょう

In [17]:
sentence = 'あえいうえおあお'
d = {}
for char in sentence:  # 文字ごとに分割
    if char in d:
        d[char] += 1
    else:  # 辞書に存在しない場合は初期値を指定
        d[char] = 1
d

{'あ': 2, 'え': 2, 'い': 1, 'う': 1, 'お': 2}

In [18]:
from collections import defaultdict

sentence = 'あえいうえおあお'
dd = defaultdict(int)  # デフォルト関数にintを指定
for char in sentence:
    dd[char] += 1  # 辞書に登録してないキーでも加算が可能
dd

defaultdict(int, {'あ': 2, 'え': 2, 'い': 1, 'う': 1, 'お': 2})

### 手順パート: 辞書データから文章を自動生成する

In [19]:
# 最初の単語の出現回数を数える

from collections import defaultdict

def get_first_words_weights(three_words_count):
    """最初の単語を選択するためのデータを作成する"""
    first_word_count = defaultdict(int)  # ❶ 値がint型のdefaultdictを作成

    for three_words, count in three_words_count.items():
        if three_words[0] == BEGIN:  # ❷ BEGINで始まるもののみを取り出す
            next_word = three_words[1]
            first_word_count[next_word] += count # ❸ 出現回数を加算
            
    return first_word_count

get_first_words_weights(three_words_count)

defaultdict(int, {'おいしい': 2, 'ビール': 1})

In [20]:
# 辞書データを単語と出現回数のリストにする

from collections import defaultdict

def get_first_words_weights(three_words_count):
    """最初の単語を選択するための辞書データを作成する"""
    first_word_count = defaultdict(int)

    for three_words, count in three_words_count.items():
        if three_words[0] == BEGIN:
            next_word = three_words[1]
            first_word_count[next_word] += count

    words = []  # ❶ 単語と重み(出現回数)を格納するリスト
    weights = []
    for word, count in first_word_count.items():
        words.append(word)  # ❷ 単語と重みをリストに追加
        weights.append(count)
            
    return words, weights

first_words, first_weights = get_first_words_weights(three_words_count)
first_words, first_weights

(['おいしい', 'ビール'], [2, 1])

In [21]:
# 文章を生成する

import random

def generate_text(fwords, fweights, markov_dict):
    """入力された辞書データを元に文章を生成する"""
    first_word = random.choices(fwords, weights=fweights)[0]  # ❶ 最初の単語を取得
    generate_words = [BEGIN, first_word]  # ❷ 文章生成用に単語を格納するリスト
    while True:
        pair = tuple(generate_words[-2:])  # ❸ 最後の2つの単語を取得
        words = markov_dict[pair]['words']  # ❹ 次の単語と重みのリストを取得
        weights = markov_dict[pair]['weights']
        next_word = random.choices(words, weights=weights)[0]  # ❺ 次の単語を取得
        if next_word == END:  # ❻ 文章が終了した場合はループを抜ける
            break
        generate_words.append(next_word)

    return ''.join(generate_words[1:])  # ❼ 単語から文章を作成

In [22]:
for _ in range(5):
    text = generate_text(first_words, first_weights, markov_dict)
    print(text)

おいしいビールを飲もう
おいしいビールを飲もう
ビールを飲もう
ビールを飲もう
おいしいビールは生


## Lesson 34 文章データを取得して前処理をしましょう

### 手順パート: 青空文庫からダウンロードした文章を前処理する

In [23]:
import requests  # ❶requestsをインポート

# 「人間失格」のファイルのURL
url = 'https://www.aozora.gr.jp/cards/000035/files/301_ruby_5915.zip'
r = requests.get(url)  # ❷ZipファイルのURLアクセス
content = r.content  # ❸Zipファイルの中身を取得

In [24]:
import io  # ❶モジュールをインポート
import zipfile

f = io.BytesIO(content)  # ❷バイナリをファイルのように変換
zipf = zipfile.ZipFile(f)  # ❸Zipファイルを開く
namelist = zipf.namelist()  # ❹ファイル一覧を取得
namelist


['ningen_shikkaku.txt']

In [25]:
data = zipf.read(namelist[0])  # ❶Zipファイルを展開しデータを取り出す
original_text = data.decode('Shift_JIS')  # ❷文字列にデコードする
print(original_text[:500])  # ❸中身を確認する

人間失格
太宰治

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）従姉妹《いとこ》

｜：ルビの付く文字列の始まりを特定する記号
（例）昔｜気質《かたぎ》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
（例）［＃３字下げ］はしがき［＃「はしがき」は大見出し］
-------------------------------------------------------

［＃３字下げ］はしがき［＃「はしがき」は大見出し］


　私は、その男の写真を三葉、見たことがある。
　一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹《いとこ》たちかと想像される）庭園の池のほとりに、荒い縞の袴《はかま》をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。醜く？　けれども、鈍い人たち（


In [26]:
import re

first_sentence = '私は、その男の写真を三葉、見たことがある。'
last_sentence = '神様みたいないい子でした'
_, text = original_text.split(first_sentence)  # ❶ 青空文庫の説明文を削除
text, _ = text.split(last_sentence)
text = first_sentence + text + last_sentence

text = text.replace('｜', '').replace('　', '')  # ❷ '｜'' と '　' を削除
text = re.sub('《\w+》', '', text)  # ❷ルビを削除
text = re.sub('［＃\w+］', '', text)  # ❷注を削除
text = text.replace('\r', '').replace('\n', '')  # ❷改行文字を削除
text = re.sub('[、「」？]', '', text)  # ❷、「」 ？を削除
text = re.sub('（\w+）', '', text)  #  ❷（）と［］で囲まれている文を削除
text = re.sub('［\w+］', '', text)

sentences = text.split('。')  # ❸。で文章を分割
print('文の数:', len(sentences))
sentences[:10]

文の数: 1177


['私はその男の写真を三葉見たことがある',
 '一葉はその男の幼年時代とでも言うべきであろうか十歳前後かと推定される頃の写真であってその子供が大勢の女のひとに取りかこまれ庭園の池のほとりに荒い縞の袴をはいて立ち首を三十度ほど左に傾け醜く笑っている写真である',
 '醜くけれども鈍い人たちは面白くも何とも無いような顔をして可愛い坊ちゃんですねといい加減なお世辞を言ってもまんざら空お世辞に聞えないくらいの謂わば通俗の可愛らしさみたいな影もその子供の笑顔に無いわけではないのだがしかしいささかでも美醜に就いての訓練を経て来たひとならひとめ見てすぐなんていやな子供だと頗る不快そうに呟き毛虫でも払いのける時のような手つきでその写真をほうり投げるかも知れない',
 'まったくその子供の笑顔はよく見れば見るほど何とも知れずイヤな薄気味悪いものが感ぜられて来る',
 'どだいそれは笑顔でない',
 'この子は少しも笑ってはいないのだ',
 'その証拠にはこの子は両方のこぶしを固く握って立っている',
 '人間はこぶしを固く握りながら笑えるものでは無いのである',
 '猿だ',
 '猿の笑顔だ']

## Lesson 35 大量の文章データから文章生成用の辞書データを生成しましょう

In [27]:
import time
from tqdm import tqdm
for i in tqdm(range(50)):  # tqdm()で囲む
    time.sleep(0.5)  # 0.5秒スリープ

100%|██████████| 50/50 [00:25<00:00,  1.98it/s]


### 手順パート: 大量の文章データからマルコフ連鎖用辞書データを自動生成する

In [28]:
from tqdm import tqdm

three_words_list = []
for sentence in tqdm(sentences):  # ❶ tqdmで進捗バーを表示する
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)
len(three_words_count)  # ❷ 3単語の組の種類を確認

100%|██████████| 1177/1177 [00:54<00:00, 21.71it/s]


30035

In [29]:
markov_dict = generate_markov_dict(three_words_count)  # ❶ 文章生成用の辞書データを作成
print(len(markov_dict))
first_words, first_weights = get_first_words_weights(three_words_count)  # ❷最初の単語と出現数を取得
print(len(first_words))

19742
498


In [30]:
for _ in range(5):
    sentence = generate_text(first_words, first_weights, markov_dict)
    print(sentence)

私もその時堀木に教えられせっせと質屋がよいをはじめ一家中が激怒してやっぱり途方にくれてヒラメの家をたずね合ったりしましたがそれでもれいの空襲で焼け出されたのです
自分は海辺の温泉地があったほど陰惨な絵が出来なかったら働いて……およそこの世で最も醜悪で下等で残酷な犯罪だとつねに彼を殺そうという覚悟は出来て悲惨な犯され方を他の者たちから窮屈でない
しかし自分は口ごもってしまいました
オントのアントは牛乳これは造血剤
自分はその時それを全く現実として受取り恐怖している


## Lesson 36 文章を自動生成するbotコマンドを作成しましょう

In [31]:
# オブジェクトをpickle化する
import pickle  # pickleモジュールをimport

data = {'name': 'takanory'}
pickled = pickle.dumps(data)  # dataオブジェクトをPickle化
pickled

b'\x80\x03}q\x00X\x04\x00\x00\x00nameq\x01X\x08\x00\x00\x00takanoryq\x02s.'

In [32]:
# pickle化されたbytesオブジェクトから元に戻す
data2 = pickle.loads(pickled)  # bytesオブジェクトをもとに戻す
data2

{'name': 'takanory'}

In [33]:
# JSONでは対応していないオブジェクトがある
import json
jsoned = json.dumps((1, 2, 3))  # タプルがリストになる
json.loads(jsoned)

[1, 2, 3]

### 手順パート: 文章生成（マルコフ）コマンドを作成する

In [34]:
import pickle

with open('markov-dict.pickle', 'wb') as f:  # ❶ファイルをバイナリ書き込みモードで開く
    data = (first_words, first_weights, markov_dict)  # ❷3つのデータをタプルにまとめる
    pickle.dump(data, f)  # ➌dataをpickle化して書き込む